In [67]:
import pandas as pd
import json
import re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [68]:
run_name = "union"
run_name = "union_mm"
run_name = "union_make_model"
run_name = "make_model_doc8643"
run_name = "make_model_doc8643_3"
run_name = "make_model_icao_code"
run_name = "union_make_model_2"
# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))


        # load JSONL
records_test = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_test.tsv") as f:
    for line in f:
        records_test.append(json.loads(line))

In [69]:
df_all = pd.DataFrame(records)
df_test = pd.DataFrame(records_test)
df_test.head()

,left,right,match,match_confidence
0,COL make VAL BEAGLE COL model VAL B206 COL nam...,COL code VAL S2T,0,0.999996
1,COL make VAL GROB COL model VAL G115 COL name ...,COL model VAL G-115D Bavarian,1,0.999994
2,COL make VAL SNOW COL model VAL 600S2C COL cer...,COL mfr VAL STINSON COL model VAL HW75,0,0.999995
3,COL make VAL CHAMPION COL model VAL 7GCBA COL ...,COL mfr VAL MOONEY COL model VAL M-20E Super 2...,0,0.999997
4,COL make VAL MURRAYAIR COL model VAL MA1 COL c...,COL code VAL MA1,1,0.999995


In [70]:
gold_test = pd.read_csv("../data/aircraft_er/" + run_name + "/test.txt", sep="\t", header=None, names=["left", "right", "gold"])

gold_all = pd.read_csv("../data/aircraft_er/" + run_name + "/all_pairs.txt", sep="\t", header=None, names=["left", "right", "gold"])
print(gold_all.head())

                                                left  \
0  COL make VAL CESSNA COL model VAL C37 COL name...   
1  COL make VAL REARWIN COL model VAL 185 COL nam...   
2  COL make VAL DOUGLAS COL model VAL DC9 COL cer...   
3  COL make VAL GULFSTREAM COL model VAL GULFSTRE...   
4  COL make VAL KAMAN COL model VAL H43 COL cert ...   

                                               right  gold  
0  COL make VAL CESSNA COL model VAL 680 COL seri...     0  
1  COL make VAL RAYTHEON COL model VAL 3000 COL s...     0  
2                                  COL code VAL E195     0  
3  COL make VAL FAIRCHILD HILLER COL model VAL FH...     0  
4             COL mfr VAL KAMAN COL model VAL HH-43B     0  


In [71]:
df_all["gold"] = gold_all["gold"]
df_test["gold"] = gold_test["gold"]

In [72]:


#Test Data

y_true_test = df_test["gold"]
y_pred_test = df_test["match"]

print("Accuracy:", accuracy_score(y_true_test, y_pred_test))
print("\nClassification report:\n", classification_report(y_true_test, y_pred_test))
print("\nConfusion matrix:\n", confusion_matrix(y_true_test, y_pred_test))



with open("../aircraft_er_predictions/" + run_name + "_metrics_test.txt", "w") as f:
    print("Accuracy:", accuracy_score(y_true_test, y_pred_test), file=f)
    print("\nClassification report:\n", file=f)
    print(classification_report(y_true_test, y_pred_test), file=f)
    print("\nConfusion matrix:\n", file=f)
    print(confusion_matrix(y_true_test, y_pred_test), file=f)

Accuracy: 0.9837120714567188

Classification report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      5749
           1       0.97      0.96      0.97      1864

    accuracy                           0.98      7613
   macro avg       0.98      0.98      0.98      7613
weighted avg       0.98      0.98      0.98      7613


Confusion matrix:
 [[5696   53]
 [  71 1793]]


In [73]:
#All Data

y_true = df_all["gold"]
y_pred = df_all["match"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))



with open("../aircraft_er_predictions/" + run_name + "_metrics_all.txt", "w") as f:
    print("Accuracy:", accuracy_score(y_true, y_pred), file=f)
    print("\nClassification report:\n", file=f)
    print(classification_report(y_true, y_pred), file=f)
    print("\nConfusion matrix:\n", file=f)
    print(confusion_matrix(y_true, y_pred), file=f)

Accuracy: 0.9962243351247072

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     54277
           1       0.99      0.99      0.99     18293

    accuracy                           1.00     72570
   macro avg       1.00      0.99      0.99     72570
weighted avg       1.00      1.00      1.00     72570


Confusion matrix:
 [[54153   124]
 [  150 18143]]


In [74]:
errors = df_test[df_test["gold"] != df_test["match"]]
print(errors[["left","right","gold","match","match_confidence"]].head(20))

                                                   left  \
30    COL make VAL RYAN COL model VAL ST COL cert VA...   
114   COL make VAL AEROSPATIALE COL model VAL SA360 ...   
230   COL make VAL LAKE COL model VAL LA4 COL name V...   
452   COL make VAL AERONCA COL model VAL 50F COL nam...   
464   COL make VAL DART COL model VAL DART GW COL ce...   
498   COL make VAL DOUGLAS COL model VAL DC3 COL cer...   
584   COL make VAL WACO COL model VAL PLA COL name V...   
925   COL make VAL GRUMMAN COL model VAL G159 COL na...   
937   COL make VAL ARROW COL model VAL SPORT COL cer...   
987   COL make VAL CESSNA COL model VAL 150 COL name...   
1021  COL make VAL CANADAIR COL model VAL CL600 2B16...   
1058  COL make VAL AIR TRACTOR COL model VAL AT302 C...   
1127  COL make VAL MOONEY COL model VAL M20L COL nam...   
1161  COL make VAL EUROCOPTER COL model VAL EC120 CO...   
1182  COL make VAL AEROSPATIALE COL model VAL AS350 ...   
1187  COL make VAL PIPER COL model VAL AE1 COL name ... 

In [75]:
errors

,left,right,match,match_confidence,gold
30,COL make VAL RYAN COL model VAL ST COL cert VA...,COL mfr VAL RYAN COL model VAL PT-20 COL code ...,0,0.744132,1
114,COL make VAL AEROSPATIALE COL model VAL SA360 ...,COL make VAL SA COL model VAL 360 COL series V...,0,0.999763,1
230,COL make VAL LAKE COL model VAL LA4 COL name V...,COL make VAL LA COL model VAL 4 COL series VAL...,0,0.947805,1
452,COL make VAL AERONCA COL model VAL 50F COL nam...,COL code VAL AC50,1,0.999991,0
464,COL make VAL DART COL model VAL DART GW COL ce...,COL mfr VAL APPLEGATE & WEYANT COL model VAL D...,0,0.997025,1
...,...,...,...,...,...
7363,COL make VAL DART COL model VAL DART GW COL ce...,COL make VAL DART COL model VAL G COL series V...,0,0.999883,1
7419,COL make VAL AEROSPATIALE COL model VAL AS355 ...,COL mfr VAL HELIBRAS COL model VAL VH-55 Esqui...,0,0.995610,1
7466,COL make VAL CUB CRAFTERS COL model VAL CC18 C...,COL code VAL AC6L,1,0.999783,0
7483,COL make VAL CONVAIR COL model VAL C131 COL na...,COL make VAL CV COL model VAL 640 COL series V...,0,0.999989,1


In [76]:
errors.to_csv("../aircraft_er_predictions/" + run_name + "_errors_review.csv", index=False)

In [77]:
def parse_record(record: str):
    """Parse Ditto serialized record into a dict of {field: value}."""
    parts = re.split(r"COL |VAL ", record.strip())
    parts = [p for p in parts if p]  # drop empties
    return {parts[i].strip(): parts[i+1].strip() for i in range(0, len(parts), 2)}

parsed = []

for _, row in errors.iterrows():
    left = parse_record(row["left"])
    right = parse_record(row["right"])
    parsed.append({
        "cictt_make": left.get("make"),
        "make": right.get("make"),
        "cictt_model": left.get("model"),
        "model": right.get("model"),
        "cictt_series": left.get("series"),
        "series": right.get("series"),
        "cictt_cert": left.get("cert"),
        "cert": right.get("cert"),
        "cictt_name": left.get("name"),
        "name": right.get("name"),
        "gold": row["gold"],
        "predicted": row["match"],
        "confidence": row["match_confidence"]
    })

aligned = pd.DataFrame(parsed)

In [78]:
aligned.to_csv("../aircraft_er_predictions/" + run_name + "_aligned_errors_review.csv", index=False)